<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/LLM/training/examples_fine_tune_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pub.towardsai.net/fine-tuning-llms-from-zero-to-hero-with-python-ollama-52258966bb6d

In [1]:

!pip install unsloth --quiet
# !pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes transformers datasets --quiet



Load the basemodel in unsloth

In [ ]:
from unsloth import FastLanguageModel

model_name = "unsloth/phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048  # Adjust based on your data length
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Add LoRA Adapter to the layers for efficient training

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank - higher = more parameters
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407
)

The training dataset that we read in looks like:

[
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
{"input","our input", "output", "expected output"},
]

In [ ]:
import json
from datasets import Dataset

with open("training_data.json", "r") as f:
    data = json.load(f)
# Format for training
def format_chat_template(item):
    return tokenizer.apply_chat_template(
        [
            {"role": "user", "content": item['input']},
            {"role": "assistant", "content": item['output']}
        ],
        tokenize=False,
        add_generation_prompt=False
    )
# Create the training dataset
formatted_data = [{"text": format_chat_template(item)} for item in data]
dataset = Dataset.from_list(formatted_data)
# Check what it looks like
print("Sample training example:")
print(formatted_data[0]["text"])

We are ready to go, so start the training !

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

training_steps=60

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=training_steps,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",  # Use "adamw_torch" if you get optimizer errors
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_steps=30,
    ),
)
# Start training! 🚀
trainer.train()

SAVE our fine-tuned model in the GGUF format which is compatible to Ollama.

In [ ]:
# save_pretrained_gguf is a unsloth function. this is ot available on standard hugging face models
model.save_pretrained_gguf(
    "fine_tuned_model",
    tokenizer,
    quantization_method="q4_k_m"  # Good balance of size/quality
)

In [ ]:
%%writefile Modelfile.py
FROM ./unsloth.Q4_K_M.gguf
PARAMETER temperature 0.1
PARAMETER top_p 0.9
PARAMETER stop ["<|endoftext|>"]
TEMPLATE "{{ .Prompt }}"
SYSTEM "You are a specialized C64 Basic coder assistant."

you can use ollama to create a model from the modelfile:

ollama create \<modelname\> -f Modelfile